In [ ]:
import kagglehub

path = kagglehub.dataset_download("gti-upm/leapgestrecog")

print("Path to dataset files:", path)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [ ]:
from PIL import Image
import os
import numpy as np

IMG_SIZE = 64
data = []
labels = []
label_map = {}
current_label = 0

DATASET_ROOT = os.path.join(path, "leapGestRecog")

print("Actual dataset folders:", os.listdir(DATASET_ROOT))

for user_folder in os.listdir(DATASET_ROOT):
    user_path = os.path.join(DATASET_ROOT, user_folder)

    if not os.path.isdir(user_path):
        continue

    print("Reading user folder:", user_folder)

    for gesture_folder in os.listdir(user_path):
        gesture_path = os.path.join(user_path, gesture_folder)

        if not os.path.isdir(gesture_path):
            continue

        if gesture_folder not in label_map:
            label_map[gesture_folder] = current_label
            current_label += 1

        label = label_map[gesture_folder]
        image_count = 0

        for img_name in os.listdir(gesture_path):
            img_path = os.path.join(gesture_path, img_name)
            try:
                img = Image.open(img_path).convert('L')
                img = img.resize((IMG_SIZE, IMG_SIZE))
                img = np.array(img)
                data.append(img)
                labels.append(label)
                image_count += 1
            except:
                continue

        print(f"  {gesture_folder}: {image_count} images loaded")


In [ ]:
X = np.array(data).reshape(-1, IMG_SIZE, IMG_SIZE, 1) / 255.0
y = to_categorical(np.array(labels))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, Dropout,
    BatchNormalization, GlobalAveragePooling2D
)
from tensorflow.keras.regularizers import l2

model = Sequential([

    Conv2D(32, (3,3), activation='relu',
           kernel_regularizer=l2(0.001),
           input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Dropout(0.3),

    Conv2D(64, (3,3), activation='relu',
           kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Dropout(0.4),

    Conv2D(128, (3,3), activation='relu',
           kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(2,2),
    Dropout(0.5),

    GlobalAveragePooling2D(),

    Dense(64, activation='relu',
          kernel_regularizer=l2(0.001)),
    Dropout(0.5),

    Dense(y.shape[1], activation='softmax')
])

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=8,
    validation_data=(X_test, y_test)
)

In [ ]:
loss, acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", acc)

In [ ]:
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title("Hand Gesture Recognition Accuracy")
plt.show()

In [ ]:
model.save("gesture_model.h5")

In [ ]:
import tensorflow as tf
model = tf.keras.models.load_model("gesture_model.h5")

In [ ]:
inv_label_map = {v: k for k, v in label_map.items()}

In [ ]:
import cv2
import numpy as np
from PIL import Image

IMG_SIZE = 64

cap = cv2.VideoCapture(0)

print("Press Q to quit")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Resize to model input
    img = cv2.resize(gray, (IMG_SIZE, IMG_SIZE))
    img = img.reshape(1, IMG_SIZE, IMG_SIZE, 1) / 255.0

    # Prediction
    prediction = model.predict(img, verbose=0)
    class_index = np.argmax(prediction)
    gesture = inv_label_map[class_index]

    # Display result
    cv2.putText(
        frame,
        f"Gesture: {gesture}",
        (20, 40),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 255, 0),
        2
    )

    cv2.imshow("Hand Gesture Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()